In [131]:
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

In [132]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

In [133]:
def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    return tokens

In [134]:
import itertools

In [135]:
def get_ngrams(tokens, n=2, patterns=None, stoplist=[]):
    tokens = list(filter(lambda x: x not in stoplist, tokens))
    lemmas = [m.parse(t)[0].normal_form for t in tokens]
    ngrams = []
    for i in range(len(lemmas) - (n-1)):
        ngram = lemmas[i:i+n]
        tngram = tokens[i:i+n]
        tags = [m.parse(t)[0].tag.POS for t in tngram]
        if patterns is not None:
            if patterns == ['VERB', 'NOUN'] and tags in patterns:
                c = 0
                for t in lemmas:
                    if t == ngram[0]:
                        c += 1
                if c > 49 and ('accs' in m.parse(tokens[i+n])[0].tag or 'accs' in m.parse(tokens[i+n])[2].tag or 'accs' in m.parse(tokens[i+n])[1].tag):
                    ngrams.append(tuple(ngram))
            elif tags in patterns:
                ngrams.append(tuple(ngram))
        else:
            ngrams.append(tuple(ngram))
    return ngrams

In [ ]:
!/Users/masha/Desktop/need_for_vyshka/P_for_Proga/python/udpipe-1.2.0-bin/bin-osx/udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
/Users/masha/Desktop/need_for_vyshka/P_for_Proga/python/udpipe-1.2.0-bin/models/russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

Loading UDPipe model: done.


In [ ]:
from nltk.parse import DependencyGraph

In [ ]:
trees = []

with open('testset2.conllu') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [ ]:
print(trees[1163])

In [ ]:
col = []
for i, tree in enumerate(trees):
    try:
        for tri in DependencyGraph(trees[i], top_relation_label='root').triples():
            if tri[0][-1] == 'VERB' and 'obj' in tri[1] and tri[2][-1] == 'NOUN':
                b = [m.parse(t)[0].normal_form for t in [tri[0][0], tri[2][0]]]
                if b not in col:
                    col.append(b)
    except AssertionError:
        continue
    except TypeError:
        continue

In [ ]:
col

In [ ]:
with open('testset2.txt', 'r', encoding='utf8') as file:  
    text = file.read()  

In [ ]:
from collections import Counter

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')

In [ ]:
tokens = normalize(text)

In [ ]:
bigrams = get_ngrams(tokens, patterns=[['VERB', 'NOUN']], stoplist=stop)
c = Counter()
for n in bigrams:
    c[n] += 1

In [ ]:
c.most_common(10)

metrics

In [ ]:
import nltk
from nltk.collocations import *

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = BigramCollocationFinder.from_documents([[m.parse(t)[0].normal_form for t in tokens]])

In [ ]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

In [ ]:
finder2.nbest(bigram_measures.pmi, 20)

In [ ]:
dice = finder2.score_ngrams(bigram_measures.dice)

In [ ]:
sorted([x for x in dice if x[1] != 1.0], key=lambda x: x[1], reverse=True)[:20]

In [ ]:
nlog = []
ndice = []
npmi = []

In [ ]:
t = 100
while len(nlog) < 100:
    for b in finder2.nbest(bigram_measures.likelihood_ratio, t):
        if b not in nlog:
            if b in col:
                nlog.append(b)
    t += 20

In [ ]:
len(nlog)

In [ ]:
t = 100
while len(npmi) < 100:
    for b in finder2.nbest(bigram_measures.pmi, t):
        if b not in npmi:
            if b in col:
                npmi.append(b)
    t += 20

In [ ]:
len(npmi)

In [ ]:
t = 0
while len(ndice) < 100:
    if t == 0:
        t2 = 100
    else:
        t2 = t+20
    for b in sorted([x for x in dice if x[1] != 1.0], key=lambda x: x[1], reverse=True)[t:t2]:
        if b[0] not in ndice:
            if b[0] in col:
                ndice.append(b[0])
    if t == 0:
        t = 100
    else:
        t += 20

In [ ]:
len(ndice)

Gold

In [ ]:
nlog[:10]

In [ ]:
ndice[0:10]

In [ ]:
npmi[0:10]

In [ ]:
gold = set(ndice) & set(npmi) & set(nlog)

In [ ]:
gold

In [ ]:
with open('verb_coll.txt', 'r', encoding='utf8') as file:  
    diction = file.readlines()

In [ ]:
d = []
for i in diction:
    ts = normalize(i.replace('\n', '').split('\t')[-1])
    ls = [m.parse(t)[0].normal_form for t in ts]
    if len(ls) < 3:
        d.append(ls)

In [ ]:
d

In [ ]:
gold = []
for g in set(ndice) & set(npmi) & set(nlog):
    print(g)
    if g in d:
        gold.append(g)

In [ ]:
gold